In [ ]:
%load_ext autoreload
%autoreload 2

from typing import Tuple
import random
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from projet_sdp.generator import generate_random
from projet_sdp.model import Model
from projet_sdp.decider import Decider


In [ ]:
N = 5
E = 3
P = 3
Q = 3

In [ ]:
def generate_dataset(number_of_instances: int) -> Tuple[int, int, int]:
    for i in range(number_of_instances):

        plan = generate_random(N=N, E=E, P=P, Q=Q)
        m = Model(plan)

        m.constraint_cant_work_more_than_needed()
        m.constraint_cant_work_on_days_off()
        m.constraint_no_qual_work()
        m.constraint_one_project_per_day()
        m.determine_affecte()
        m.determine_debute()
        m.determine_realise()

        objectives = [m.objective_max_gain, m.objective_min_affecte, m.objective_min_length]
        target_objective = random.choice(objectives)
        target_objective()

        m.update()
        m.optimize(quiet=True)

        yield np.array(m.objective_values())

In [ ]:
decider = Decider()

In [ ]:
TRAIN_SIZE = 1000
TEST_SIZE = 1000

In [ ]:
X_train, X_test = list(generate_dataset(TRAIN_SIZE)), list(generate_dataset(TEST_SIZE))
Y_train, Y_test = list(map(lambda x: decider.decide(x), X_train)), list(map(lambda x: decider.decide(x), X_test))

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
print(np.count_nonzero(Y_train), "of", TRAIN_SIZE, "pass")

In [ ]:
model = LogisticRegression()
model = model.fit(X_train, Y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
print(classification_report(Y_test, Y_pred))